# Conversion Tools

https://forum.image.sc/t/converting-whole-slide-images-to-ome-tiff-a-new-workflow/32110/4

<img src="blog-2019-12-converting-whole-slide-images.jpg" style="height:300px" />



## Basics

The two basic commands are `bioformats2raw` and `raw2ometiff`. Together they provide a pipeline to scalably convert large images into OME-TIFF. The primary caveat is that they require **twice** the storage for the conversion.

In [1]:
%%time
!bioformats2raw --help

Missing required parameters: <inputPath>, <outputPath>
Usage: <main class> [--debug] [--extra-readers[=<extraReaders>[,
                    <extraReaders>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tStringArgsCsv>] [-c=<compressionType>]
                    [--compression-parameter=<compressionParameter>]
                    [--dimension-order=<dimensionOrder>]
                    [--file_type=<fileType>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>] [--pyramid-name=<pyramidName>]
                    [-r=<pyramidResolutions>]
                    [--scale-format-string=<scaleFormatString>]
                    [-w=<tileWidth>] <inputPath> <outputPath>
      <inputPath>    file to convert
      <outputPath>   path to the output pyramid directory
      --additional-scale-format-string-args=<additionalScaleFormatStringArgsCsv>
                    

In [18]:
%%time
!raw2ometiff --help

Missing required parameters: <inputDirectory>, <outputFilePath>
Usage: <main class> [--debug] [--legacy] [--rgb] [--compression=<compression>]
                    [--max_workers=<maxWorkers>] <inputDirectory>
                    <outputFilePath>
      <inputDirectory>   Directory containing pixel data to convert
      <outputFilePath>   Relative path to the output OME-TIFF file
      --compression=<compression>
                         Compression type for output OME-TIFF file (Uncompressed,
                           LZW, JPEG-2000, JPEG-2000 Lossy, JPEG, zlib; default: LZW)
      --debug            Turn on debug logging
      --legacy           Write a Bio-Formats 5.9.x pyramid instead of OME-TIFF
      --max_workers=<maxWorkers>
                         Maximum number of workers (default: 16)
      --rgb              Attempt to write channels as RGB; channel count must be 3
CPU times: user 8.31 ms, sys: 8.46 ms, total: 16.8 ms
Wall time: 711 ms


## Simple Invocation

Here we will use Bio-Formats ability to generate test data in order to quickly test within the notebook.

See https://docs.openmicroscopy.org/bio-formats/latest/developers/generating-test-images.html for more information.

**First we generate the raw intermediate format:**

In [19]:
%%time
!bioformats2raw a.fake /tmp/output-1

2020-05-20 12:03:14,418 [main] INFO  loci.formats.ImageReader - FakeReader initializing a.fake
2020-05-20 12:03:14,547 [main] INFO  c.g.bioformats2raw.Converter - Using 2 pyramid resolutions
2020-05-20 12:03:14,547 [main] INFO  c.g.bioformats2raw.Converter - Preparing to write pyramid sizeX 512 (tileWidth: 1024) sizeY 512 (tileWidth: 1024) imageCount 1
2020-05-20 12:03:14,797 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileWidth to 512
2020-05-20 12:03:14,797 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileHeight to 512
2020-05-20 12:03:14,809 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 0, 0, 0] to /0/0
2020-05-20 12:03:14,820 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 1/1
2020-05-20 12:03:14,820 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1589968994809] time[11] tag[getTile]
2020-05-20 12:03:14,823 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - successfully 

**Then we convert that output into an OME-TIFF:**

In [10]:
%%time
!raw2ometiff /tmp/output-1 /tmp/output-1.ome.tiff

2020-05-20 11:27:01,619 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Creating tiled pyramid file /tmp/output-1.ome.tiff
2020-05-20 11:27:01,673 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Number of resolution levels: 2
2020-05-20 11:27:01,682 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 0
2020-05-20 11:27:01,683 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 1
2020-05-20 11:27:01,684 [main] INFO  org.perf4j.TimingLogger - start[1589966821362] time[321] tag[initialize]
2020-05-20 11:27:01,736 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Converting resolution #0
2020-05-20 11:27:01,739 [main] INFO  org.perf4j.TimingLogger - start[1589966821737] time[2] tag[getInputTileBytes]
2020-05-20 11:27:01,741 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Converting resolution #1
2020-05-20 11:27:01,742 [main] INFO  org.perf4j.TimingLogger - start[1589966821741] time[1] tag[getInputTileBytes]
2020-05-20 11:27:01,750 [poo

**The same operation on larger generated data, still proceeds timely:**

In [7]:
%%time
!bioformats2raw 'big&sizeX=10000&sizeY=10000.fake' /tmp/output-2

2020-05-20 11:26:22,230 [main] INFO  loci.formats.ImageReader - FakeReader initializing big&sizeX=10000&sizeY=10000.fake
2020-05-20 11:26:22,366 [main] INFO  c.g.bioformats2raw.Converter - Using 7 pyramid resolutions
2020-05-20 11:26:22,366 [main] INFO  c.g.bioformats2raw.Converter - Preparing to write pyramid sizeX 10000 (tileWidth: 1024) sizeY 10000 (tileWidth: 1024) imageCount 1
2020-05-20 11:26:22,604 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [6, 0, 0, 0, 0] to /0/0
2020-05-20 11:26:22,624 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - tile read complete 1/100
2020-05-20 11:26:22,624 [pool-1-thread-7] INFO  org.perf4j.TimingLogger - start[1589966782604] time[20] tag[getTile]
2020-05-20 11:26:22,628 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - successfully wrote at [6, 0, 0, 0, 0] to /0/0
2020-05-20 11:26:22,629 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=6144 y

2020-05-20 11:26:22,773 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - tile read complete 14/100
2020-05-20 11:26:22,773 [pool-1-thread-6] INFO  org.perf4j.TimingLogger - start[1589966782768] time[4] tag[getTile]
2020-05-20 11:26:22,774 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - successfully wrote at [7, 2, 0, 0, 0] to /0/0
2020-05-20 11:26:22,774 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=7168 yy=2048 width=1024 height=1024
2020-05-20 11:26:22,776 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [8, 2, 0, 0, 0] to /0/0
2020-05-20 11:26:22,782 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - tile read complete 15/100
2020-05-20 11:26:22,782 [pool-1-thread-6] INFO  org.perf4j.TimingLogger - start[1589966782776] time[6] tag[getTile]
2020-05-20 11:26:22,784 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - successfully wrote at [8, 2, 0, 0, 0] to /0/0
2020-05-2

2020-05-20 11:26:22,877 [pool-1-thread-13] INFO  c.g.bioformats2raw.Converter - tile read complete 32/100
2020-05-20 11:26:22,877 [pool-1-thread-13] INFO  org.perf4j.TimingLogger - start[1589966782871] time[5] tag[getTile]
2020-05-20 11:26:22,878 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - tile read complete 33/100
2020-05-20 11:26:22,878 [pool-1-thread-7] INFO  org.perf4j.TimingLogger - start[1589966782872] time[5] tag[getTile]
2020-05-20 11:26:22,879 [pool-1-thread-13] INFO  c.g.bioformats2raw.Converter - successfully wrote at [3, 4, 0, 0, 0] to /0/0
2020-05-20 11:26:22,879 [pool-1-thread-13] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=3072 yy=4096 width=1024 height=1024
2020-05-20 11:26:22,880 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - successfully wrote at [4, 4, 0, 0, 0] to /0/0
2020-05-20 11:26:22,880 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=4096

2020-05-20 11:26:22,924 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - tile read complete 44/100
2020-05-20 11:26:22,924 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1589966782914] time[10] tag[getTile]
2020-05-20 11:26:22,924 [pool-1-thread-16] INFO  c.g.bioformats2raw.Converter - successfully wrote at [5, 1, 0, 0, 0] to /0/0
2020-05-20 11:26:22,924 [pool-1-thread-16] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=5120 yy=1024 width=1024 height=1024
2020-05-20 11:26:22,925 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - tile read complete 45/100
2020-05-20 11:26:22,925 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [2, 5, 0, 0, 0] to /0/0
2020-05-20 11:26:22,925 [pool-1-thread-15] INFO  org.perf4j.TimingLogger - start[1589966782914] time[11] tag[getTile]
2020-05-20 11:26:22,925 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=20

2020-05-20 11:26:22,975 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 8, 0, 0, 0] to /0/0
2020-05-20 11:26:22,975 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=0 yy=8192 width=1024 height=1024
2020-05-20 11:26:22,975 [pool-1-thread-16] INFO  c.g.bioformats2raw.Converter - tile read complete 69/100
2020-05-20 11:26:22,975 [pool-1-thread-16] INFO  org.perf4j.TimingLogger - start[1589966782970] time[5] tag[getTile]
2020-05-20 11:26:22,976 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [3, 8, 0, 0, 0] to /0/0
2020-05-20 11:26:22,976 [pool-1-thread-16] INFO  c.g.bioformats2raw.Converter - successfully wrote at [1, 8, 0, 0, 0] to /0/0
2020-05-20 11:26:22,976 [pool-1-thread-16] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=1024 yy=8192 width=1024 height=1024
2020-05-20 11:26:22,977 [pool-1-thread-16] INFO  c.g.bioforma

2020-05-20 11:26:23,032 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [8, 5, 0, 0, 0] to /0/0
2020-05-20 11:26:23,037 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - tile read complete 90/100
2020-05-20 11:26:23,037 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1589966783032] time[5] tag[getTile]
2020-05-20 11:26:23,039 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [8, 5, 0, 0, 0] to /0/0
2020-05-20 11:26:23,039 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=8192 yy=5120 width=1024 height=1024
2020-05-20 11:26:23,053 [pool-1-thread-11] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [1, 7, 0, 0, 0] to /0/0
2020-05-20 11:26:23,054 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [4, 7, 0, 0, 0] to /0/0
2020-05-20 11:26:23,059 [pool-1-thread-11] INFO  c.g.bioformats2raw.Converter - tile read c

2020-05-20 11:26:23,504 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1589966783494] time[10] tag[getTileDownsampled]
2020-05-20 11:26:23,504 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - tile read complete 5/25
2020-05-20 11:26:23,504 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1589966783494] time[10] tag[getTile]
2020-05-20 11:26:23,506 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - successfully wrote at [4, 3, 0, 0, 0] to /0/1
2020-05-20 11:26:23,506 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=1 plane=0 xx=4096 yy=3072 width=904 height=1024
2020-05-20 11:26:23,507 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 4, 0, 0, 0] to /0/1
2020-05-20 11:26:23,516 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1589966783507] time[9] tag[getTileDownsampled]
2020-05-20 11:26:23,516 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - tile read complete 6/25

2020-05-20 11:26:23,732 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [3, 1, 0, 0, 0] to /0/1
2020-05-20 11:26:23,732 [pool-1-thread-10] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [2, 2, 0, 0, 0] to /0/1
2020-05-20 11:26:23,733 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 0, 0, 0] to /0/1
2020-05-20 11:26:23,749 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1589966783732] time[17] tag[getTileDownsampled]
2020-05-20 11:26:23,749 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - tile read complete 16/25
2020-05-20 11:26:23,749 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1589966783732] time[17] tag[getTile]
2020-05-20 11:26:23,749 [pool-1-thread-10] INFO  org.perf4j.TimingLogger - start[1589966783732] time[16] tag[getTileDownsampled]
2020-05-20 11:26:23,749 [pool-1-thread-10] INFO  c.g.bioformats2raw.Converter - tile read complete 17/25
2020-05-20 11:26:23,749

2020-05-20 11:26:24,358 [pool-1-thread-16] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [2, 0, 0, 0, 0] to /0/2
2020-05-20 11:26:24,359 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [2, 2, 0, 0, 0] to /0/2
2020-05-20 11:26:24,362 [pool-1-thread-8] INFO  org.perf4j.TimingLogger - start[1589966784359] time[2] tag[getTileDownsampled]
2020-05-20 11:26:24,362 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - tile read complete 2/9
2020-05-20 11:26:24,362 [pool-1-thread-8] INFO  org.perf4j.TimingLogger - start[1589966784359] time[2] tag[getTile]
2020-05-20 11:26:24,363 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - successfully wrote at [2, 2, 0, 0, 0] to /0/2
2020-05-20 11:26:24,363 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=2 plane=0 xx=2048 yy=2048 width=452 height=452
2020-05-20 11:26:24,364 [pool-1-thread-16] INFO  org.perf4j.TimingLogger - start[1589966784358] time[

2020-05-20 11:26:24,991 [pool-1-thread-9] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [1, 1, 0, 0, 0] to /0/3
2020-05-20 11:26:24,992 [pool-1-thread-9] INFO  org.perf4j.TimingLogger - start[1589966784991] time[1] tag[getTileDownsampled]
2020-05-20 11:26:24,993 [pool-1-thread-9] INFO  c.g.bioformats2raw.Converter - tile read complete 4/4
2020-05-20 11:26:24,993 [pool-1-thread-9] INFO  org.perf4j.TimingLogger - start[1589966784991] time[1] tag[getTile]
2020-05-20 11:26:24,993 [pool-1-thread-9] INFO  c.g.bioformats2raw.Converter - successfully wrote at [1, 1, 0, 0, 0] to /0/3
2020-05-20 11:26:24,993 [pool-1-thread-9] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=3 plane=0 xx=1024 yy=1024 width=226 height=226
2020-05-20 11:26:24,993 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileWidth to 625
2020-05-20 11:26:24,993 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileHeight to 625
2020-05-20 11:26:24,995 [pool-

**But of course produces larger output:**

In [11]:
%%time
!raw2ometiff /tmp/output-2 /tmp/output-2.ome.tiff

2020-05-20 11:27:11,341 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Creating tiled pyramid file /tmp/output-2.ome.tiff
2020-05-20 11:27:11,401 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Number of resolution levels: 7
2020-05-20 11:27:11,414 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 0
2020-05-20 11:27:11,415 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 1
2020-05-20 11:27:11,415 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 2
2020-05-20 11:27:11,415 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 3
2020-05-20 11:27:11,415 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 4
2020-05-20 11:27:11,415 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 5
2020-05-20 11:27:11,415 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 6
2020-05-20 11:27:11,417 [main] I

2020-05-20 11:27:11,740 [pool-1-thread-12] INFO  org.perf4j.TimingLogger - start[1589966831650] time[90] tag[writeTile]
2020-05-20 11:27:11,742 [main] INFO  org.perf4j.TimingLogger - start[1589966831740] time[1] tag[getInputTileBytes]
2020-05-20 11:27:11,745 [pool-1-thread-10] INFO  org.perf4j.TimingLogger - start[1589966831666] time[78] tag[writeTile]
2020-05-20 11:27:11,746 [main] INFO  org.perf4j.TimingLogger - start[1589966831745] time[1] tag[getInputTileBytes]
2020-05-20 11:27:11,747 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1589966831663] time[84] tag[writeTile]
2020-05-20 11:27:11,760 [main] INFO  org.perf4j.TimingLogger - start[1589966831759] time[1] tag[getInputTileBytes]
2020-05-20 11:27:11,761 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1589966831643] time[118] tag[writeTile]
2020-05-20 11:27:11,762 [pool-1-thread-8] INFO  org.perf4j.TimingLogger - start[1589966831673] time[89] tag[writeTile]
2020-05-20 11:27:11,762 [pool-1-thread-3] INFO  org.perf4

2020-05-20 11:27:11,964 [main] INFO  org.perf4j.TimingLogger - start[1589966831962] time[1] tag[getInputTileBytes]
2020-05-20 11:27:11,983 [main] INFO  org.perf4j.TimingLogger - start[1589966831964] time[18] tag[getInputTileBytes]
2020-05-20 11:27:11,984 [main] INFO  org.perf4j.TimingLogger - start[1589966831983] time[0] tag[getInputTileBytes]
2020-05-20 11:27:11,985 [main] INFO  org.perf4j.TimingLogger - start[1589966831984] time[0] tag[getInputTileBytes]
2020-05-20 11:27:11,986 [main] INFO  org.perf4j.TimingLogger - start[1589966831985] time[0] tag[getInputTileBytes]
2020-05-20 11:27:11,987 [main] INFO  org.perf4j.TimingLogger - start[1589966831986] time[0] tag[getInputTileBytes]
2020-05-20 11:27:11,988 [main] INFO  org.perf4j.TimingLogger - start[1589966831987] time[0] tag[getInputTileBytes]
2020-05-20 11:27:12,043 [pool-1-thread-8] INFO  org.perf4j.TimingLogger - start[1589966831939] time[103] tag[writeTile]
2020-05-20 11:27:12,045 [main] INFO  org.perf4j.TimingLogger - start[15899

2020-05-20 11:27:12,223 [pool-1-thread-15] INFO  org.perf4j.TimingLogger - start[1589966832175] time[48] tag[writeTile]
2020-05-20 11:27:12,224 [main] INFO  org.perf4j.TimingLogger - start[1589966832223] time[1] tag[getInputTileBytes]
2020-05-20 11:27:12,225 [main] INFO  org.perf4j.TimingLogger - start[1589966832224] time[0] tag[getInputTileBytes]
2020-05-20 11:27:12,225 [pool-1-thread-6] INFO  org.perf4j.TimingLogger - start[1589966832209] time[16] tag[writeTile]
2020-05-20 11:27:12,226 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1589966832209] time[17] tag[writeTile]
2020-05-20 11:27:12,225 [main] INFO  org.perf4j.TimingLogger - start[1589966832225] time[0] tag[getInputTileBytes]
2020-05-20 11:27:12,227 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Converting resolution #4
2020-05-20 11:27:12,227 [main] INFO  org.perf4j.TimingLogger - start[1589966832227] time[0] tag[getInputTileBytes]
2020-05-20 11:27:12,227 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Converting re

In [13]:
!ls -ltrah /tmp/output*tiff

-rw-r--r--  1 jamoore  wheel    54K May 20 11:27 /tmp/output-1.ome.tiff
-rw-r--r--  1 jamoore  wheel    20M May 20 11:27 /tmp/output-2.ome.tiff


## Compression

Both commands additionally provide additional arguments like `--compression` which you can experiment with:

**Here we leave the OME-TIFF uncompressed to see how much larger it will be:**

In [16]:
%%time
!raw2ometiff /tmp/output-2 /tmp/output-2.ome.tiff --compression=Uncompressed

2020-05-20 11:29:23,157 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Creating tiled pyramid file /tmp/output-2.ome.tiff
2020-05-20 11:29:23,213 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Number of resolution levels: 7
2020-05-20 11:29:23,225 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 0
2020-05-20 11:29:23,226 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 1
2020-05-20 11:29:23,226 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 2
2020-05-20 11:29:23,226 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 3
2020-05-20 11:29:23,226 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 4
2020-05-20 11:29:23,226 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 5
2020-05-20 11:29:23,226 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 6
2020-05-20 11:29:23,228 [main] I

2020-05-20 11:29:23,354 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1589966963353] time[0] tag[writeTile]
2020-05-20 11:29:23,356 [main] INFO  org.perf4j.TimingLogger - start[1589966963353] time[2] tag[getInputTileBytes]
2020-05-20 11:29:23,356 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1589966963356] time[0] tag[writeTile]
2020-05-20 11:29:23,358 [main] INFO  org.perf4j.TimingLogger - start[1589966963356] time[1] tag[getInputTileBytes]
2020-05-20 11:29:23,358 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1589966963358] time[0] tag[writeTile]
2020-05-20 11:29:23,359 [main] INFO  org.perf4j.TimingLogger - start[1589966963358] time[1] tag[getInputTileBytes]
2020-05-20 11:29:23,360 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1589966963360] time[0] tag[writeTile]
2020-05-20 11:29:23,361 [main] INFO  org.perf4j.TimingLogger - start[1589966963360] time[1] tag[getInputTileBytes]
2020-05-20 11:29:23,362 [pool-1-thread-5] INFO  org.perf4j.TimingLog

2020-05-20 11:29:23,455 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1589966963455] time[0] tag[writeTile]
2020-05-20 11:29:23,456 [main] INFO  org.perf4j.TimingLogger - start[1589966963455] time[0] tag[getInputTileBytes]
2020-05-20 11:29:23,456 [pool-1-thread-6] INFO  org.perf4j.TimingLogger - start[1589966963456] time[0] tag[writeTile]
2020-05-20 11:29:23,457 [main] INFO  org.perf4j.TimingLogger - start[1589966963456] time[0] tag[getInputTileBytes]
2020-05-20 11:29:23,457 [pool-1-thread-7] INFO  org.perf4j.TimingLogger - start[1589966963457] time[0] tag[writeTile]
2020-05-20 11:29:23,457 [main] INFO  org.perf4j.TimingLogger - start[1589966963457] time[0] tag[getInputTileBytes]
2020-05-20 11:29:23,458 [pool-1-thread-9] INFO  org.perf4j.TimingLogger - start[1589966963457] time[0] tag[writeTile]
2020-05-20 11:29:23,459 [main] INFO  org.perf4j.TimingLogger - start[1589966963457] time[1] tag[getInputTileBytes]
2020-05-20 11:29:23,460 [pool-1-thread-8] INFO  org.perf4j.TimingLog

2020-05-20 11:29:23,510 [main] INFO  org.perf4j.TimingLogger - start[1589966963228] time[281] tag[convertToPyramid]

real	0m1.183s
user	0m2.279s
sys	0m0.289s


In [17]:
!ls -ltrah /tmp/output*tiff

-rw-r--r--  1 jamoore  wheel    54K May 20 11:27 /tmp/output-1.ome.tiff
-rw-r--r--  1 jamoore  wheel   138M May 20 11:29 /tmp/output-2.ome.tiff


### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.